In [ ]:
#Question 1 part -A

In [ ]:
import numpy as np
import pandas as pd

dice_rolls = pd.read_csv('/content/hmm_pb1.csv', header=None)

stat = [1, 2]
init_prob = np.array([0.5, 0.5])
trans_pb = np.array([
    [0.95, 0.05],
    [0.05, 0.95],
])
ems_pb = np.array([
    [1/6]*5 + [1/6],
    [1/10]*5 + [1/2],
])
observations = dice_rolls.iloc[0].values - 1

n_stat = len(stat)
n_observations = len(observations)
V = np.zeros((n_stat, n_observations))
backpointer = np.zeros((n_stat, n_observations), dtype=int)

for a in range(n_stat):
    V[a, 0] = np.log(init_prob[a]) + np.log(ems_pb[a, observations[0]])


for t in range(1, n_observations):
    for a in range(n_stat):
        transition_pb = V[:, t-1] + np.log(trans_pb[:, a])
        V[a, t] = np.max(transition_pb) + np.log(ems_pb[a, observations[t]])
        backpointer[a, t] = np.argmax(transition_pb)

b_path = [np.argmax(V[:, n_observations - 1])]
for t in range(n_observations - 1, 0, -1):
    b_path.insert(0, backpointer[b_path[0], t])

b_path_states = [stat[state] for state in b_path]
sequence_y = ''.join(map(str, b_path_states))
for i in range(0, len(sequence_y), 15):
    print(' '.join(sequence_y[i:i+15]))


1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 2 2 2 2
2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1 1 2 2 2 2 2 2
2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
2 2 2 2 2 2 2 2 2 2 1 1 1 1 1
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


In [ ]:
#Question 1 part-b

In [ ]:
observations = dice_rolls.values.squeeze() - 1

def forward(observations, stat, init_prob, trans_pb, ems_pb):
    alpha = np.zeros((len(observations), len(stat)))
    c = np.zeros(len(observations))
    alpha[0, :] = init_prob * ems_pb[:, observations[0]]
    c[0] = 1.0 / np.sum(alpha[0, :])
    alpha[0, :] *= c[0]

    for t in range(1, len(observations)):
        for j in range(len(stat)):
            alpha[t, j] = alpha[t-1].dot(trans_pb[:, j]) * ems_pb[j, observations[t]]
        c[t] = 1.0 / np.sum(alpha[t, :])
        alpha[t, :] *= c[t]

    return alpha, c

def backward(observations, stat, trans_pb, ems_pb, c):
    beta = np.zeros((len(observations), len(stat)))

    beta[-1, :] = c[-1]

    for t in reversed(range(len(observations) - 1)):
        for i in range(len(stat)):
            beta[t, i] = (trans_pb[i, :] * ems_pb[:, observations[t + 1]] * beta[t + 1, :]).sum()
        beta[t, :] *= c[t]

    return beta
alpha, c = forward(observations, stat, init_prob, trans_pb, ems_pb)
beta = backward(observations, stat, trans_pb, ems_pb, c)

alpha_ratio = alpha[137, 0] / alpha[137, 1]
beta_ratio = beta[137, 0] / beta[137, 1]

print(f'α1_138/α2_138: {alpha_ratio}')
print(f'β1_138/β2_138: {beta_ratio}')


α1_138/α2_138: 9.795724733488365
β1_138/β2_138: 5.761919016897878


In [ ]:
#Question 2

In [ ]:
def forward(observations, stat, start_pb, trans_prob, em_pb):
    alpha = np.zeros((len(observations), stat))
    c = np.zeros(len(observations))
    alpha[0, :] = start_pb * em_pb[:, observations[0]]
    c[0] = 1.0 / np.sum(alpha[0, :])
    alpha[0, :] *= c[0]

    for t in range(1, len(observations)):
        for j in range(stat):
            alpha[t, j] = alpha[t-1].dot(trans_prob[:, j]) * em_pb[j, observations[t]]
        c[t] = 1.0 / np.sum(alpha[t, :])
        alpha[t, :] *= c[t]

    return alpha, c

def backward(observations, stat, trans_prob, em_pb, c):
    beta = np.zeros((len(observations), stat))

    beta[-1, :] = c[-1]

    for t in reversed(range(len(observations) - 1)):
        for i in range(stat):
            beta[t, i] = (trans_prob[i, :] * em_pb[:, observations[t + 1]] * beta[t + 1, :]).sum()
        beta[t, :] *= c[t]

    return beta

def baum_welch(observations, n_stat, n_iter=100):
    n_symbols = np.max(observations) + 1
    n_observations = len(observations)

    start_pb = np.random.rand(n_stat)
    start_pb /= start_pb.sum()
    trans_prob = np.random.rand(n_stat, n_stat)
    trans_prob /= trans_prob.sum(axis=1, keepdims=True)
    em_pb = np.random.rand(n_stat, n_symbols)
    em_pb /= em_pb.sum(axis=1, keepdims=True)
    k=range(n_stat)
    for i in range(n_iter):
        alpha, c = forward(observations, len(k), start_pb, trans_prob, em_pb)
        beta = backward(observations, len(k), trans_prob, em_pb, c)

        gamma = alpha * beta
        gamma /= gamma.sum(axis=1, keepdims=True)

        xi = np.zeros((n_observations - 1, n_stat, n_stat))
        for t in range(n_observations - 1):
            denom = np.sum(alpha[t, :] * trans_prob * em_pb[:, observations[t + 1]].T * beta[t + 1, :])
            for i in range(n_stat):
                for j in range(n_stat):
                    xi[t, i, j] = alpha[t, i] * trans_prob[i, j] * em_pb[j, observations[t + 1]] * beta[t + 1, j] / denom

        start_prob = gamma[0, :]
        trans_prob = np.sum(xi, axis=0) / np.sum(gamma[:-1, :], axis=0).reshape(-1, 1)
        em_pb.fill(0)
        for t in range(n_observations):
            em_pb[:, observations[t]] += gamma[t, :]
        em_pb /= gamma.sum(axis=0).reshape(-1, 1)

    return start_pb, trans_prob, em_pb

observations = pd.read_csv('/content/hmm_pb2.csv', header=None).values.squeeze() - 1

n_stat = 2
int_prob, trans_prob, em_pb = baum_welch(observations, n_stat, n_iter=100)

print("Inti_Prob", int_prob)
print("trans_prob", trans_prob)
print("em_pb", em_pb)


Inti_Prob [0.03418643 0.96581357]
trans_prob [[0.203846   0.79726234]
 [0.79949517 0.20167715]]
em_pb [[0.1629864  0.17944403 0.19166148 0.16054405 0.12148477 0.18387927]
 [0.18073768 0.15532325 0.14847995 0.23335472 0.13182926 0.15027514]]
